In [5]:
!pip install future

    100% |################################| 829kB 1.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /root/.cache/pip/wheels/c2/50/7c/0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
Successfully built future


In [7]:
import builtins

In [8]:
from future.utils import listitems

In [21]:
%%writefile pip_tools.py

import copy
import os
import sys
import argparse

def combine_requirements(files, output_path='/tmp/requirements.txt'):
    """
    @params files: list of filenames in requirements.txt format
    @returns: union, intersection, duplicates, version_diffs
    """
    lines = {}
    union = {}
    intersection = {}
    duplicates = {}
    version_diffs = {}
    file_versions_map = {}
    for _file in files:
        print('_file', _file)
        with open(_file, 'r') as f:
            _file = os.path.basename(_file)
            file_versions_map[_file] = {}
            lines[_file] = f.readlines()
            for line in lines[_file]:
                if line.startswith('#') or (len(line.strip()) == 0):
                    print('skipping line with contents {}'.format(line))
                    continue
                try:
                    package, version = get_package_info(line)
                except ValueError as err:
                    raise(ValueError(" : ".join([err.message, line])))
                existing_list = union.get(package, [])
                new_list = copy.copy(existing_list)
                new_list.append((version, _file))
                union.update({package: new_list})
                file_versions_map[_file].update({package: (version, line)})
                for (existing_version, existing_file) in existing_list:
                    print(existing_version, existing_file)
                    if existing_version == version:
                        if package not in duplicates:
                            duplicates[package] = [(existing_version, existing_file), (version, _file)]
                        else:
                            #print(duplicates.keys())
                            duplicates.get(package).append((version, _file))
                    else:
                        if version_diffs.get(package, None):
                            version_diffs.get(package, []).append((version, _file))
                        else:            
                            version_diffs[package] = [(existing_version, existing_file), (version, _file)]
                                                           
    intersection = set(union.keys())
    # conservative intersection, do not include same package with different versions
    for _file in files:
        _file = os.path.basename(_file)
        intersection = intersection.intersection(set(file_versions_map[_file].keys()))
        orig_intersection = copy.copy(intersection)
        # remove packages with different versions
        for package, info in file_versions_map[_file].items():
            version, line = info
            if package in orig_intersection.intersection(set(version_diffs.keys())):
                print("Removing package {} with different versions {}".format(
                    package, version_diffs[package]))
                intersection.remove(package)
                                                           

    out_lines = []
    for _file in file_versions_map.keys():
        out_lines.append('\n# ' + _file + '\n\n')
        for package, info in file_versions_map[_file].items():
            version, line = info
            if package in duplicates.keys():
                out_lines.append('# ' + line.strip() +  ' (duplicate)\n')
            elif package in version_diffs.keys():
                out_lines.append('# ' + line.strip() + ' (version_diff)\n')
            else:
                out_lines.append(line)
        
    with open(output_path, 'w') as f:
        f.writelines(out_lines)
        
    return union, intersection, version_diffs, duplicates
                
        
def get_package_info(line):
    """
    @param line: a line from a requirements.txt file
    WARNING: currently only supports '==' not '<=, ~=, etc'
    examples:
      foo==1.0.1
      foo
      [-e] git+git://git.myproject.org/SomeProject#egg=SomeProject
      [-e] git+https://git.myproject.org/SomeProject#egg=SomeProject
      [-e] git+ssh://git.myproject.org/SomeProject#egg=SomeProject
      [-e] git://git.myproject.org/MyProject.git@master#egg=MyProject
      [-e] git://git.myproject.org/MyProject.git@v1.0#egg=MyProject
      [-e] git://git.myproject.org/MyProject.git@da39a3ee5e6b4b0d3255bfef95601890afd80709#egg=MyProject
    """
    if line.startswith('-e') or line.startswith('[-e'):
        if line.find('#egg=') != -1:
            prefix, package = line.strip().split('#egg=')
            # set a default version
            version = 'master-latest'
        else:
            raise ValueError("unrecognized format for line {}\n expected '#egg=<python-package>'".format(line))
        if prefix.find('@') != -1:
            version = prefix.split('@')[1]
    else:
        split = line.split('==')
        package = split[0].strip()
        if len(split) == 2:
            version = split[1].strip()
        elif len(split) == 1:
            version = 'latest'
        else:
            raise(ValueError('unrecognized requirements.txt line {}'.format(line)))
    if len(package) == 0:
        raise(ValueError('un-recognized version for line {}'.format(line)))
    return package, version
            
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("method")
    parser.add_argument("-f", "--files", help="comma separated no spaces list of files to process")
    parser.add_argument("-o", "--output-path",
                        help="comma separated no spaces list of files to process",
                        default="/tmp/requirements.txt")
    args = parser.parse_args()    
        
    if args.method == 'combine_requirements':
        files = args.files.split(',')
        combine_requirements(files, output_path=args.output_path)
    
if __name__ == '__main__':
    main()
    

Overwriting pip_tools.py


In [15]:
base_dir = "/notebooks/projects/core/Haystack"
print(base_dir)
files = [os.path.join(base_dir, x) for x in os.listdir(base_dir) if x.endswith('requirements.txt') ]
print(files)

/notebooks/projects/core/Haystack
['/notebooks/projects/core/Haystack/django_requirements.txt', '/notebooks/projects/core/Haystack/scipy_requirements.txt', '/notebooks/projects/core/Haystack/clients_requirements.txt']


In [16]:
union, intersection, version_diffs, duplicates = combine_requirements(files)
import pprint
print('union', len(union))
pprint.pprint(union)
print('intersection', len(intersection))
pprint.pprint(intersection)
print('version_diffs')
pprint.pprint(version_diffs)
print('duplicates')
pprint.pprint(duplicates)

skipping line with contents # Most django requirements currently reside in 

skipping line with contents # docker-stacks/base_requirements.txt

skipping line with contents 

skipping line with contents #redis_opentracing

('b83674f3267310c95aa24dc47b6f495705ad8c65', 'django_requirements.txt')
Removing package django_opentracing with different versions [('b83674f3267310c95aa24dc47b6f495705ad8c65', 'django_requirements.txt'), ('latest', 'clients_requirements.txt')]
('union', 38)
{'Cython': [('0.26', 'scipy_requirements.txt')],
 'django_opentracing': [('b83674f3267310c95aa24dc47b6f495705ad8c65',
                         'django_requirements.txt'),
                        ('latest', 'clients_requirements.txt')],
 'flower': [('latest', 'django_requirements.txt')],
 'ipykernel': [('4.6.1', 'scipy_requirements.txt')],
 'ipython': [('latest', 'scipy_requirements.txt')],
 'ipython-genutils': [('0.2.0', 'scipy_requirements.txt')],
 'ipywidgets': [('6.0.0', 'scipy_requirements.txt')],
 'jaeger-cl

In [60]:
env = """mysql_root_password=hs4demo
mysql_user=root
mysql_pass=hsabi
mysql_monitor_user=monitor_user
mysql_monitor_pass=akble902aj3
haystack_user=hsdemo
haystack_password=fever4me
logzio_auth_token=ulkhtivrLZOuyGMrzOGUvOFfACyogNov
mysql_data_path=~/mysqldata
logzio_monitor_path=~/.logzio/monitor
logzio_logs_path=~/.logzio/logs
redis_data_path=~/.redis/data
core_haystack_path=~/projects/core/Haystack
"""

In [137]:
b1 = [1,2,3,4,5,9,11,15]
b2 = [4,5,6,7,8]
set(b1).intersection(b2)

{4, 5}

In [138]:
b1

[1, 2, 3, 4, 5, 9, 11, 15]

In [10]:
splits = env.split('\n')
print(splits)`````````````````a
x = [ x.split('=')[0] for x in splits]
print(x)
print("\n".join(x))

['mysql_root_password=hs4demo', 'mysql_user=root', 'mysql_pass=hsabi', 'mysql_monitor_user=monitor_user', 'mysql_monitor_pass=akble902aj3', 'haystack_user=hsdemo', 'haystack_password=fever4me', 'logzio_auth_token=ulkhtivrLZOuyGMrzOGUvOFfACyogNov', 'mysql_data_path=~/mysqldata', 'logzio_monitor_path=~/.logzio/monitor', 'logzio_logs_path=~/.logzio/logs', 'redis_data_path=~/.redis/data', 'core_haystack_path=~/projects/core/Haystack', '']
['mysql_root_password', 'mysql_user', 'mysql_pass', 'mysql_monitor_user', 'mysql_monitor_pass', 'haystack_user', 'haystack_password', 'logzio_auth_token', 'mysql_data_path', 'logzio_monitor_path', 'logzio_logs_path', 'redis_data_path', 'core_haystack_path', '']
mysql_root_password
mysql_user
mysql_pass
mysql_monitor_user
mysql_monitor_pass
haystack_user
haystack_password
logzio_auth_token
mysql_data_path
logzio_monitor_path
logzio_logs_path
redis_data_path
core_haystack_path

